# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load the csv exported in Part I to a DataFrame
city_data = ("../output_data/cities.csv")
city_data_df = pd.read_csv(city_data)
city_data_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Qaanaaq,77.48,-69.36,47.23,59,88,2.91,GL,1594592385
1,Halifax,44.65,-63.57,73.00,77,9,3.00,CA,1594592822
2,Thompson,55.74,-97.86,75.20,50,75,9.17,CA,1594592394
3,New Norfolk,-42.78,147.06,37.99,92,0,2.44,AU,1594592371
4,Pathein,16.78,94.73,79.38,87,100,6.38,MM,1594592916


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
fig = gmaps.figure(zoom_level=2,center=(20,40))
locations = city_data_df[["Latitude","Longitude"]]

# city_data_df = city_data_df.dropna()
humidity = city_data_df["Humidity"]

In [4]:
heatmap = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity = 10, point_radius=1)
fig.add_layer(heatmap)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cities_narrow = city_data_df.loc[(city_data_df["Max Temp"]>70) & (city_data_df["Max Temp"]<80)&(city_data_df["Wind Speed"]<10)&(city_data_df["Cloudiness"]==0),:]
cities_narrow = cities_narrow.dropna(how="any")    
cities_narrow                                                                                       

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
100,Lata,40.16,-8.33,75.99,58,0,1.99,PT,1594592473
136,Mitsamiouli,-11.38,43.28,77.36,72,0,3.62,KM,1594592957
157,Kiryat Gat,31.61,34.76,75.99,72,0,1.01,IL,1594592433
195,Zhezkazgan,47.80,67.71,78.66,49,0,8.75,KZ,1594592992
213,Buritis,-15.62,-46.42,70.36,40,0,3.27,BR,1594592999
278,Rafrāf,37.19,10.18,75.20,60,0,4.70,TN,1594593025
390,Angoche,-16.23,39.91,71.19,83,0,6.82,MZ,1594593072
465,Arys,42.43,68.80,75.20,31,0,4.47,KZ,1594593104
487,Misratah,32.38,15.09,76.39,77,0,7.63,LY,1594593113


In [6]:
len(cities_narrow)

9

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = cities_narrow.copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(100)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
100,Lata,40.16,-8.33,75.99,58,0,1.99,PT,1594592473,
136,Mitsamiouli,-11.38,43.28,77.36,72,0,3.62,KM,1594592957,
157,Kiryat Gat,31.61,34.76,75.99,72,0,1.01,IL,1594592433,
195,Zhezkazgan,47.80,67.71,78.66,49,0,8.75,KZ,1594592992,
213,Buritis,-15.62,-46.42,70.36,40,0,3.27,BR,1594592999,
278,Rafrāf,37.19,10.18,75.20,60,0,4.70,TN,1594593025,
390,Angoche,-16.23,39.91,71.19,83,0,6.82,MZ,1594593072,
465,Arys,42.43,68.80,75.20,31,0,4.47,KZ,1594593104,
487,Misratah,32.38,15.09,76.39,77,0,7.63,LY,1594593113,


In [8]:
 # Use the search Hotel Name
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# params dictionary to update each iteration

params = {"radius": 5000,
          "type": "lodging",
          "keyword": "hotel",
          "key": g_key}

# Use the lat/lng we recovered to identify Hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"

    # make request and print url
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing Hotel information.....skip")
        pass

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, 
              row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers_lon_lat = hotel_df[["Latitude", "Longitude"]]
fig = gmaps.figure()
markers = gmaps.marker_layer(markers_lon_lat,info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))